In [ ]:
import cv2
import dlib
import face_recognition
import json
import os
import numpy as np

# 加載會員資料庫
def load_members():
    members_file = os.path.join(os.getcwd(), 'members.json')
    with open('members.json', 'r') as file:
        data = json.load(file)
        # 轉換編碼為 numpy array
        for member in data['members']:
            member['encoding'] = np.array(member['encoding'])
        return data

def save_members(data):
    # 轉換編碼為列表格式以便 JSON 支持
    for member in data['members']:
        if isinstance(member['encoding'], np.ndarray):
            member['encoding'] = member['encoding'].tolist()
    members_file = os.path.join(os.getcwd(), 'members.json')        
    with open('members.json', 'w') as file:
        json.dump(data, file)

members_data = load_members()

# 解析會員資料
member_encodings = []
member_names = []
member_status = []

for member in members_data['members']:
    encoding = member.get('encoding', None)
    if encoding is not None:  # 確保 encoding 不為 None
        member_encodings.append(np.array(encoding))  # 確保編碼是 numpy array
        member_names.append(member.get('member_name', 'Unknown'))
        member_status.append(True)

# 初始化攝像頭
cap = cv2.VideoCapture(0)
detector = dlib.get_frontal_face_detector()
font = cv2.FONT_HERSHEY_SIMPLEX

# 檢查影像是否模糊
def is_image_blurry(image, threshold=100):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()
    return laplacian_var < threshold

# 註冊新會員
def register_new_member(face_encoding, full_frame):
    member_name = input("請輸入您的姓名: ")
    is_member = input("是否加入會員? (yes/no): ").lower() == 'yes'
    member_gender = input("請輸入您的性別: ")
    member_phone = input("請輸入您的電話: ")
    member_age = input("請輸入您的年齡: ")
    member_city = input("請輸入您的城市: ")
    photo_filename = os.path.join(os.getcwd(), f"{member_name}.jpg")

    if is_member:
        new_member = {
            "member_name": member_name,
            "member_gender": member_gender,
            "member_phone": member_phone,
            "member_age": member_age,
            "member_city": member_city,
            "encoding": face_encoding.tolist()  # 保存為列表
        }
        members_data["members"].append(new_member)
        save_members(members_data)

        global member_encodings, member_names, member_status
        member_encodings.append(face_encoding)
        member_names.append(new_member["member_name"])
        member_status.append(True)
        
        # 儲存會員全臉照片
        cv2.imwrite(photo_filename, full_frame)
        print("新會員註冊成功！照片已儲存。")
    else:
        print("照片已刪除。")

# 處理每幀影像
def process_frame(frame):
    if is_image_blurry(frame):
        cv2.putText(frame, "Image is blurry. Please adjust.", (50, 50), font, 0.7, (0, 0, 255), 2)
        cv2.imshow('Face Recognition', frame)
        return True  # 影像模糊，繼續捕捉影像

    dlib_faces = detector(frame, 0)
    face_locations = face_recognition.face_locations(frame)
    face_encodings = face_recognition.face_encodings(frame, face_locations)

    if len(face_encodings) == 0:
        cv2.putText(frame, "No face detected. Please adjust.", (50, 50), font, 0.7, (0, 0, 255), 2)
        cv2.imshow('Face Recognition', frame)
        return True  # 沒有偵測到人臉，繼續捕捉影像

    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # 確認 Dlib 檢測到人臉
        if any(left <= d.left() <= right and top <= d.top() <= bottom for d in dlib_faces):
            matches = face_recognition.compare_faces(member_encodings, face_encoding)
            name = "Unknown"
            member = False

            if True in matches:
                first_match_index = matches.index(True)
                name = member_names[first_match_index]
                member = member_status[first_match_index]
            else:
                # 拍攝全臉照片
                full_face_image = frame.copy()

                # 偵測到未知人臉，提示註冊
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
                cv2.putText(frame, "Unknown - Register?", (left, top - 10), font, 0.5, (0, 0, 255), 2)
                cv2.imshow('Face Recognition', frame)
                cv2.waitKey(0)

                # 提示註冊選項
                register_choice = input("偵測到未知人臉。是否註冊為新會員？ (yes/no): ").lower()
                if register_choice == 'yes':
                    # 暫停影像捕捉並註冊新會員
                    register_new_member(face_encoding, full_face_image)
                    return False  # 註冊完成，通知需要重新啟動攝像頭
                else:
                    # 選擇不註冊，刪除照片並提示
                    del full_face_image
                    print("照片已刪除。")
                    continue  # 繼續捕捉影像

            # 畫出人臉框和會員信息
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(frame, name, (left, top - 10), font, 0.5, (255, 255, 255), 2)
            member_info = "Member" if member else "Non-Member"
            cv2.putText(frame, member_info, (left, bottom + 20), font, 0.5, (255, 255, 255), 2)

    return True  # 繼續捕捉影像

while True:
    ret, frame = cap.read()
    if not ret or cv2.waitKey(1) & 0xFF == 27:
        break

    if not process_frame(frame):
        # 註冊新會員後，重新啟動攝像頭
        cap.release()
        cv2.destroyAllWindows()
        cv2.waitKey(1)
        cap = cv2.VideoCapture(0)
        continue

    cv2.imshow('Face Recognition', frame)

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)